In [1]:
# This program aims to decode Reference Pool
# Procesing workflow:
# 1. Identify and exclude PCR primers. In Reference Sequences, we used universal primer sets.
# 2. RS correcting the Reference Sequence and extract Reference Sequence.
# 3. Get Reference Number (the first four bases after forward primer) and Segment Number (the following four bases after sequence of Reference Number )
# 4. Put together Reference Strands into Reference Pool.

from reedsolo import RSCodec
# 2 error correction codes
rsc = RSCodec(2)

def converter(seq):
    converter = {'A': '00', 'C': '01', 'G': '10', 'T': '11'} 
    bases = list(seq) 
    bases = [converter[base] for base in bases] 
    return ''.join(bases)

def deconverter(seq):
    deconverter = {'00': 'A', '01': 'C', '10': 'G', '11': 'T'} 
    doubleBits = [seq[i:i+2] for i in range(0, len(seq), 2)]
    doubleBits = [deconverter[doubleBit] for doubleBit in doubleBits] 
    return ''.join(doubleBits)

def oligoToBase3(seq):
    oligoToBase3 = {'G': '0', 'T': '1', 'A': '2', 'C': '3', 'N': '3'}
    oligoToBase3Converted = []
    for i in range (0, len(seq)):
        if (oligoToBase3[seq[i]] == '3'):
            return -1
        else:
            oligoToBase3Converted.append(oligoToBase3[seq[i]])
            if seq[i] == 'C':
                oligoToBase3 = {'G': '0', 'T': '1', 'A': '2', 'C': '3', 'N': '3'}
            elif seq[i] == 'G':
                oligoToBase3 = {'T': '0', 'A': '1', 'C': '2', 'G': '3', 'N': '3'}
            elif seq[i] == 'T':
                oligoToBase3 = {'A': '0', 'C': '1', 'G': '2', 'T': '3', 'N': '3'}
            elif seq[i] == 'A':
                oligoToBase3 = {'C': '0', 'G': '1', 'T': '2', 'A': '3', 'N': '3'}


    return ''.join(oligoToBase3Converted)


def ternaryToDecimal(n):
    decimal = 0
    n = ''.join(reversed(n))
    for i in range (0, len(n)):
        decimal += (int(n[i]))*(pow(3, i))
    return decimal


def most_frequent(List):
    counter = 0
    string = List[0]
     
    for i in List:
        curr_frequency = List.count(i)
        if(curr_frequency> counter):
            counter = curr_frequency
            string = i
            
    return string


%store -r array_reference
# Using readlines() to get the content of every line of FASTQ file.
file_R1 = open('sample5_S5_L001_R1_001.fastq', 'r')
file_R2 = open('sample5_S5_L001_R2_001.fastq', 'r')

Lines_R1 = file_R1.readlines()
Lines_R2 = file_R2.readlines()

new_Lines_R1 = []
new_Lines_R2 = []

complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}

# Pick up sequences from FASTQ file and preserve the first 151 nt in case some oligo lengths were beyond 151 nt.
for i in range (0, int(len(Lines_R1))):
    if i%4 == 1:
        new_Lines_R1.append(Lines_R1[i][0:151].strip('\n'))

for i in range (0, int(len(Lines_R2))):
    if i%4 == 1:
        reverse_complement_R2 = "".join(complement.get(base, base) for base in reversed(Lines_R2[i][0:151].strip('\n')))
        new_Lines_R2.append(reverse_complement_R2)
        
print('Total number of reads: ' + str(len(new_Lines_R1)))      

new_Lines_Combo = []

# Define a list to store Read Number of those reads which passed the following conditions:
read_index_1 = []

for i in range (0, int(len(new_Lines_R1))):
    # Because the length of Reference Strand is less than 151 nt, we only need to find if 142 nt of the forward read and reverse read overlap
    if (new_Lines_R1[i][0:142] == new_Lines_R2[i][9:151]):  # This judgement is to ensure that the paired-end read generates consistent results, therefore enhance the probability that this read is valid.
        new_Lines_Combo.append(new_Lines_R1[i][0:142])
        read_index_1.append(i)
    
Lines = new_Lines_Combo
print('Total number of reads after processing: ' + str(len(Lines)))

# Get the number of valid reads
res_read_index_1 = [*set(read_index_1)]
        
primerLength = 21
segmentLimit = 9
referenceLimit = 16
referenceStrands = [[None]*segmentLimit for _ in range(referenceLimit)]
candi_referenceStrands = [ [ [] for i in range(segmentLimit) ] for i in range(referenceLimit) ]

read_index_2 = []
read_index_3 = []

read_number = -1

for reference in Lines: 
    read_number += 1
    arrangedReference = []
    reference = reference.strip('\n')   # In '.txt' files, there may be '\n' symbols meaning the start of a new line. Those symbols needs to be eliminated. 
    # Exclude forward and reverse primers from Reference Strands
    reference = reference[primerLength:len(reference)-primerLength]

    RS = reference[-12:]      # Extract RS Sequence (base-3)

    reference = reference[0:(len(reference)-12)]
    compare_candidate = reference[8:]
    
    for m in range (0, referenceLimit):
        for n in range (0, segmentLimit):
            if compare_candidate == array_reference[m][n]:
                read_index_2.append(read_index_1[read_number])   # Store Read Number with successful decoding attempt before RS correction

    # Convert RS Sequence to ternary number, then to binary number
    RS_segment_binary_total = []
    for i in range (0, int(len(RS)/6)):
        RS_segment = RS[i*6:(i+1)*6]
        RS_segment_base3 = oligoToBase3(RS_segment)
        if (RS_segment_base3 == -1):
            break
        RS_segment_decimal = ternaryToDecimal(RS_segment_base3)
        RS_segment_binary = bin(RS_segment_decimal)
        RS_segment_binary = RS_segment_binary[2:]
        RS_segment_binary = '0'*(8-len(RS_segment_binary)) + RS_segment_binary
        RS_segment_binary = str(RS_segment_binary)
        RS_segment_binary_total.append(RS_segment_binary)
    

    RS_segment_binary_total = ''.join(RS_segment_binary_total)
    if (len(RS_segment_binary_total) != referenceLimit):
        continue
        
    binaryConverted = converter(reference)   # Convert Reference Sequence to binary number 
    binaryConverted = binaryConverted + RS_segment_binary_total   # Combine the reference and RS code in binary form

    binaryList = [int(binaryConverted[i:i + 8], 2) for i in range(0, len(binaryConverted), 8)]
    bytesList = bytes(binaryList)
    try:
        RSDecoded = rsc.decode(bytesList)[0]   # RS correction
    except:
        continue
    
    bytes_as_bits = ''.join(format(byte, '08b') for byte in RSDecoded)
    baseDeconverted = deconverter(bytes_as_bits)   # Convert corrected bytes back to DNA sequences 

    referenceSequence = baseDeconverted[0:4]    # Get Reference Number
    segmentSequence = baseDeconverted[4:8]      # Get Segment Number
    referenceBase3 = oligoToBase3(referenceSequence)
    if (referenceBase3 == -1):
        continue
    segmentBase3 = oligoToBase3(segmentSequence)
    if(segmentBase3 == -1):
        continue

    referenceBase10 = ternaryToDecimal(referenceBase3)
    segmentBase10 = ternaryToDecimal(segmentBase3)
    
    baseDeconverted = baseDeconverted[8:]   # Get Reference Sequence

    if (len(baseDeconverted) != 80):
        continue
        
    for m in range (0, referenceLimit):
        for n in range (0, segmentLimit):
            if baseDeconverted == array_reference[m][n]:
                read_index_3.append(read_index_1[read_number])   # Store Read Number with successful decoding attempt before RS correction
    try:
        candi_referenceStrands[referenceBase10][segmentBase10].append(baseDeconverted) 
        
    except:
        continue
        
for m in range (0, referenceLimit):
    for n in range (0, segmentLimit):
        if (referenceStrands[m][n] == None):
            try:
                referenceStrands[m][n] = most_frequent(candi_referenceStrands[m][n])  # Only keep the sequence appearing most frequently
            except:
                pass
        
# Due to repeated "CGTA" unkts in the Reference Strands, some sequences may not be decodable even if we enhance the sequencing coverage. In this case, a remedial mechanism may need to be introduced.        
# To determine the unique repeat sequence in one Reference Segment (first 4-letter decodeable unit)


# Get the number of successfully decoded reads before and after RS correction
res_read_index_2 = [*set(read_index_2)]
res_read_index_3 = [*set(read_index_3)]
print('Before Remedy: ') 
print('Total number of valid reads: ' + str(len(res_read_index_1)))
print('Exact matched sequences before RS correction: ' + str(len(res_read_index_2)))
print('Matched sequences after RS correction: ' + str(len(res_read_index_3)))

# Get the index list of non-decodable reads
whole_read = list(range(len(new_Lines_R1)))
not_decodable_index = [element for element in whole_read if element not in res_read_index_2]

# Define correctable remedy reads
remedy_reads = []
# Define a list to store Read Number of those reads which passed the following conditions
read_index_4 = []   

candi_referenceStrands = [ [ [] for i in range(segmentLimit) ] for i in range(referenceLimit) ]
    
for i in range(0, len(not_decodable_index)):
    candidate = new_Lines_R1[not_decodable_index[i]]  # Pick up candidate reads according to the read index
    for j in range (0, len(candidate)):
        if candidate[0:21] == 'TCAACTGGTGATTCGTGCAAC' and candidate[j:j+21] == 'ACGGTAGCTTCCTGTATGCCT':  # Recognition of reverse primer sequence
            remedy_reads.append(candidate[0:j+21])  
            read_index_4.append(not_decodable_index[i])
            
# Supplement remedy reads to a length of 80 nt            
remedy_reads_corrected = []
for reference in remedy_reads:
    
    reference = reference[21:len(reference)-21]  # Exclude PCR forward and reverse primers
    RS = reference[-12:]
    referenceSequence = reference[0:4]
    segmentSequence = reference[4:8]
    reference = reference[8:len(reference)-12]   # Get payload of Reference
    
    # Base supplementing rule
    if (len(reference) == 0):
        reference = 20 * 'CGTA'
        
    if len(reference)%4 == 0:
        reference = reference + reference[-4:]*(int((80-len(reference))/4))
        
    elif len(reference)%4 == 1:
        if (reference[-1:]) == "C":
            reference = reference + 'GTA'
            reference = reference + reference[-4:]*(int((80-len(reference))/4))
        elif (reference[-1:]) == "G":
            reference = reference + 'TAC'
            reference = reference + reference[-4:]*(int((80-len(reference))/4))
        elif (reference[-1:]) == "T":
            reference = reference + 'ACG'
            reference = reference + reference[-4:]*(int((80-len(reference))/4))
        elif (reference[-1:]) == "A":
            reference = reference + 'CGT'
            reference = reference + reference[-4:]*(int((80-len(reference))/4))
            
    elif len(reference)%4 == 2:
        if (reference[-2:]) == "CG":
            reference = reference + 'TA'
            reference = reference + reference[-4:]*(int((80-len(reference))/4))
        elif (reference[-2:]) == "GT":
            reference = reference + 'AC'
            reference = reference + reference[-4:]*(int((80-len(reference))/4))
        elif (reference[-2:]) == "TA":
            reference = reference + 'CG'
            reference = reference + reference[-4:]*(int((80-len(reference))/4))
        elif (reference[-2:]) == "AC":
            reference = reference + 'GT'
            reference = reference + reference[-4:]*(int((80-len(reference))/4))
        
    elif len(reference)%4 == 3:
        if (reference[-3:]) == "CGT":
            reference = reference + 'A'
            reference = reference + reference[-4:]*(int((80-len(reference))/4))
        elif (reference[-3:]) == "GTA":
            reference = reference + 'C'
            reference = reference + reference[-4:]*(int((80-len(reference))/4))
        elif (reference[-3:]) == "TAC":
            reference = reference + 'G'
            reference = reference + reference[-4:]*(int((80-len(reference))/4))
        elif (reference[-3:]) == "ACG":
            reference = reference + 'T'
            reference = reference + reference[-4:]*(int((80-len(reference))/4))
    
    if (len(reference) == 80):
        reference = referenceSequence + segmentSequence + reference + RS
        remedy_reads_corrected.append(reference)
        
# Go back to normal processing 
read_index_5 = []
read_index_6 = []

read_number = -1

for reference in remedy_reads_corrected:
    read_number += 1
    RS = reference[-12:]      # Extract RS Sequence (base-3)

    reference = reference[0:(len(reference)-12)]
    
    compare_candidate = reference[8:]
    
    for m in range (0, referenceLimit):
        for n in range (0, segmentLimit):
            if compare_candidate == array_reference[m][n]:
                read_index_5.append(read_index_4[read_number])   # Store Read Number with successful decoding attempt before RS correction

    # Convert RS Sequence to ternary number, then to binary number
    RS_segment_binary_total = []
    for i in range (0, int(len(RS)/6)):
        RS_segment = RS[i*6:(i+1)*6]
        RS_segment_base3 = oligoToBase3(RS_segment)
        if (RS_segment_base3 == -1):
            break
        RS_segment_decimal = ternaryToDecimal(RS_segment_base3)
        RS_segment_binary = bin(RS_segment_decimal)
        RS_segment_binary = RS_segment_binary[2:]
        RS_segment_binary = '0'*(8-len(RS_segment_binary)) + RS_segment_binary
        RS_segment_binary = str(RS_segment_binary)
        RS_segment_binary_total.append(RS_segment_binary)
    

    RS_segment_binary_total = ''.join(RS_segment_binary_total)
    if (len(RS_segment_binary_total) != referenceLimit):
        continue
        
    binaryConverted = converter(reference)   # Convert Reference Sequence to binary number 
    binaryConverted = binaryConverted + RS_segment_binary_total   # Combine the reference and RS code in binary form

    binaryList = [int(binaryConverted[i:i + 8], 2) for i in range(0, len(binaryConverted), 8)]
    bytesList = bytes(binaryList)
    try:
        RSDecoded = rsc.decode(bytesList)[0]   # RS correction
    except:
        continue
    
    bytes_as_bits = ''.join(format(byte, '08b') for byte in RSDecoded)
    baseDeconverted = deconverter(bytes_as_bits)   # Convert corrected bytes back to DNA sequences 

    referenceSequence = baseDeconverted[0:4]    # Get Reference Number
    segmentSequence = baseDeconverted[4:8]      # Get Segment Number
    referenceBase3 = oligoToBase3(referenceSequence)
    if (referenceBase3 == -1):
        continue
    segmentBase3 = oligoToBase3(segmentSequence)
    if (segmentBase3 == -1):
        continue

    referenceBase10 = ternaryToDecimal(referenceBase3)
    segmentBase10 = ternaryToDecimal(segmentBase3)
    
    baseDeconverted = baseDeconverted[8:]   # Get Reference Sequence

    if (len(baseDeconverted) != 80):
        continue
        
    for m in range (0, referenceLimit):
        for n in range (0, segmentLimit):
            if baseDeconverted == array_reference[m][n]:
                read_index_6.append(read_index_4[read_number])   # Store Read Number with successful decoding attempt before RS correction
    try:
        if (referenceStrands[referenceBase10][segmentBase10] == None):
            candi_referenceStrands[referenceBase10][segmentBase10].append(baseDeconverted) # Store the RS-corrected sequence in candidate sequence list
    
    except:
        continue


for i in range (0, referenceLimit):
    for j in range (0, segmentLimit):
        if (referenceStrands[i][j] == None):
            referenceStrands[i][j] = most_frequent(candi_referenceStrands[i][j])  # Only keep the sequence appearing most frequently
            
  
for i in range (0, len(referenceStrands)):
    try:
        referenceStrands[i] = ''.join(referenceStrands[i])    # Put together reference strands
    except:
        pass

print(referenceStrands)

%store referenceStrands      
%store -r trimmed_whole_array
# Compare to standard 
for i in range (0, len(trimmed_whole_array)):
    if (trimmed_whole_array[i] == referenceStrands[i]):
        print("Reference #" + str(i) + ': Yes!')
    else:
        print("Reference #" + str(i) + ': No!')   
    
# Get the number of successfully decoded reads before and after RS correction
res_read_index_5 = [*set(read_index_5)]
res_read_index_6 = [*set(read_index_6)]
print('Remedy Process: ')
print('Total number of valid reads: ' + str(len(read_index_4)))
print('Exact matched sequences before RS correction: ' + str(len(res_read_index_5)))
print('Matched sequences after RS correction: ' + str(len(res_read_index_6)))       

print('After Remedy: ')    
print('Total number of valid reads: ' + str(len(res_read_index_1) + len(read_index_4)))
print('Exact matched sequences before RS correction: ' + str(len(res_read_index_2) + len(res_read_index_5)))
print('Matched sequences after RS correction: ' + str(len(res_read_index_3) + len(res_read_index_6)))


Total number of reads: 4133
Total number of reads after processing: 2678
Before Remedy: 
Total number of valid reads: 2678
Exact matched sequences before RS correction: 2573
Matched sequences after RS correction: 2649
['CGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACCGACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTAGTAAGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACCTACGTACGTACGTACGTACGTACGTACGACT', 'ACGTACGTACGTACTTCCGTCCGTACGTAGGTACGTACGTACGTACGAACGTACGT